In [1]:
!nvidia-smi

Thu Jun 30 14:47:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install datasets
!pip install transformers
!pip install rouge-score
!pip install nltk

from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset, load_metric
import torch, random
import nltk
from nltk import sent_tokenize

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
nltk.download('punkt')

True

In [6]:
rouge = load_metric("rouge", seed=2002)

In [7]:
model_checkpoint = 'facebook/bart-large-xsum'

In [10]:
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

model = BartForConditionalGeneration.from_pretrained(model_checkpoint)
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((102

In [ ]:
model.num_parameters()

In [11]:
data_test = load_dataset('xsum', split='test')

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [12]:
batch_size = 8

In [13]:
# map data correclty
def generate_summary(batch):
    inputs = tokenizer(batch['document'],
                       padding=True,
                       truncation=True,
                       max_length=1024,
                       add_special_tokens=False,
                       return_tensors='pt')
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # The following decoding parameters are set according to the BART's config.json file
    outputs = model.generate(input_ids,
                             attention_mask=attention_mask,
                             min_length=11,
                             max_length=62,
                             num_beams=6,
                             no_repeat_ngram_size=3,
                             length_penalty=1.0,
                             early_stopping=True
                             )

    # all special tokens will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch['pred'] = output_str

    return batch

In [14]:
results = data_test.map(generate_summary,
                        batched=True,
                        batch_size=batch_size,
                        remove_columns=['document', 'id'])

labels = results['summary']
predictions = results['pred']

Parameter 'function'=<function generate_summary at 0x7fe5eab3de60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1417 [00:00<?, ?ba/s]

In [17]:
# ROUGE expects a newline after each sentence
clean_preds = ["\n".join(sent_tokenize(pred.replace('[X_SEP]', ' '))) for pred in predictions]
clean_labels = [label.replace(" .", ".") for label in labels]

for i in range(len(predictions)):
    print(f"Item {i}:")
    print("Label:")
    print(clean_labels[i])
    print("\n")
    print("Prediction:")
    print(clean_preds[i])
    print("\n")

Streaming output truncated to the last 5000 lines.
Prediction:
The Syrian government has accused the US and other Western powers of failing to understand that the country is "in war".


Item 10779:
Label:
A seven-year-old girl drowned at her best friend's birthday pool party while there was no lifeguard on duty, an inquest heard.


Prediction:
The father of a six-year-old girl who drowned at a holiday park has told an inquest there were no lifeguards on duty.


Item 10780:
Label:
Worcestershire's Brett D'Oliveira made his first hundred in over a year as he shared a 243-run opening stand with fellow centurion Daryl Mitchell against Derbyshire on day three at Derby.


Prediction:
Worcestershire opener Brett D'Oliveira and batsman Mitch Mitchell both hit centuries on day three against Derbyshire at New Road.


Item 10781:
Label:
India's dramatic move to scrap 500 ($7.60) and 1,000 rupee notes is poor economics, a leading economist says.


Prediction:
India's "demonetisation" of 500 and 1,

In [20]:
rouge_output = rouge.compute(predictions=clean_preds, references=clean_labels, use_stemmer=True)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(rouge_output[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)

print(rouge_dict)

{'rouge1': 45.15, 'rouge2': 22.09, 'rougeL': 36.89, 'rougeLsum': 36.9}
